In [11]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [12]:
import pandas as pd

path = r"G:\My Drive\Github\py-2025-epge-dados1-finalassignment\data\input\resultados_merged.parquet"

df = pd.read_parquet(path)
df.shape

(27928, 56)

In [13]:
# Passo 1 — Definir o target (y)

y = np.log(df["valor_oferta"])


In [14]:
#2.1 Criar variáveis derivadas mínimas

# 2.1 Criar variáveis derivadas mínimas

df["bairro_std"] = df["bairro"].astype(str).str.upper().str.strip()
df.loc[df["bairro"].isna(), "bairro_std"] = "MISSING"

df["cidade_std"] = df["cidade.x"].astype(str).str.upper().str.strip()
df.loc[df["cidade.x"].isna(), "cidade_std"] = "MISSING"

df["uf_cidade"] = df["uf.x"].astype(str) + "_" + df["cidade_std"].astype(str)
df["uf_cidade_bairro"] = df["uf_cidade"] + "_" + df["bairro_std"]

# filtrar bairros raros (ex: >= 15)
counts = df["uf_cidade_bairro"].value_counts()
valid = counts[counts >= 15].index

df["uf_cidade_bairro_f"] = np.where(df["uf_cidade_bairro"].isin(valid),
                                    df["uf_cidade_bairro"],
                                    "OUTROS_BAIRROS")

# Converter data_licitacao para datetime (tenta ISO e depois dia/mês/ano)
d = pd.to_datetime(df["data_licitacao"], errors="coerce")
if d.isna().mean() > 0.5:
    d = pd.to_datetime(df["data_licitacao"], errors="coerce", dayfirst=True)

df["data_licitacao_dt"] = d
df["ano"] = df["data_licitacao_dt"].dt.year
df["mes"] = df["data_licitacao_dt"].dt.month

count_cols = [ "area_total", "area_privativa", "area_terreno", "quartos", "salas", "vagas_garagem", 
              "lavabos", "suites", "cozinhas", "varandas", "sacadas", "terracos", "areas_servico", 
              "dce", "churrasqueiras", "wc", "wc_emp"
]

for c in count_cols:
    df[f"{c}_missing"] = df[c].isna().astype(int)

# 2.2 Cenários de modelagem (somente mudando FEATURES e cat_cols)

BASE_CAT = ["uf.x", "modalidade_de_venda.x", "tipo"]
CIDADE = ["uf_cidade"]
BAIRRO = ["uf_cidade_bairro_f"]

BASE_NUM = [
    "ano", "mes",

    "area_total", "area_total_missing",
    "area_privativa", "area_privativa_missing",
    "area_terreno", "area_terreno_missing",

    "quartos", "quartos_missing",
    "salas", "salas_missing",
    "vagas_garagem", "vagas_garagem_missing",
    "lavabos", "lavabos_missing",
    "suites", "suites_missing",

    "cozinhas", "cozinhas_missing",
    "varandas", "varandas_missing",
    "sacadas", "sacadas_missing",
    "terracos", "terracos_missing",
    "areas_servico", "areas_servico_missing",
    "dce", "dce_missing",
    "churrasqueiras", "churrasqueiras_missing",
    "wc", "wc_missing",
    "wc_emp", "wc_emp_missing",
]

PRICE_ANCHORS = ["preco", "valor_de_avaliacao"]

SCENARIOS = {
    # modelo estrutural (sem âncoras monetárias)
    "structural_uf": {
        "modelselected": "CatBoostRegressor",
        "features": BASE_NUM + BASE_CAT,
        "cat_cols": BASE_CAT
    },
    "structural_city": {
        "modelselected": "CatBoostRegressor",
        "features": BASE_NUM + BASE_CAT + CIDADE,
        "cat_cols": BASE_CAT + CIDADE
    },
    "structural_bairro": {
        "modelselected": "CatBoostRegressor",
        "features": BASE_NUM + BASE_CAT + CIDADE + BAIRRO,
        "cat_cols": BASE_CAT + CIDADE + BAIRRO
    },

    # upper bound preditivo (com âncoras monetárias)
    "predictive_uf": {
        "modelselected": "CatBoostRegressor",
        "features": PRICE_ANCHORS + BASE_NUM + BASE_CAT,
        "cat_cols": BASE_CAT
    },
    "predictive_city": {
        "modelselected": "CatBoostRegressor",
        "features": PRICE_ANCHORS + BASE_NUM + BASE_CAT + CIDADE,
        "cat_cols": BASE_CAT + CIDADE
    },
    "predictive_bairro": {
        "modelselected": "CatBoostRegressor",
        "features": PRICE_ANCHORS + BASE_NUM + BASE_CAT + CIDADE + BAIRRO,
        "cat_cols": BASE_CAT + CIDADE + BAIRRO
    },
}

ACTIVE_SCENARIO = "structural_uf"  # <-- troque aqui quando quiser

FEATURES = SCENARIOS[ACTIVE_SCENARIO]["features"]
cat_cols = SCENARIOS[ACTIVE_SCENARIO]["cat_cols"]
modelselected = SCENARIOS[ACTIVE_SCENARIO]["modelselected"]

X = df[FEATURES].copy()
X.shape

(27928, 39)

In [15]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

def run_linear_baseline(df, features, cat_cols, test_year=2024):
    X = df[features].copy()
    y = np.log(df["valor_oferta"])

    is_test = (df["ano"] == test_year)
    X_train, X_test = X[~is_test], X[is_test]
    y_train, y_test = y[~is_test], y[is_test]

    from sklearn.impute import SimpleImputer

    num_cols = [c for c in features if c not in cat_cols]

    preprocessor = ColumnTransformer(
        transformers=[
            # categóricas: imputar missing -> depois one-hot
            ("cat", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
            ]), cat_cols),

            # numéricas: imputar missing -> depois passa
            ("num", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="median"))
            ]), num_cols),
        ],
        remainder="drop"
    )

    model = Pipeline([
        ("prep", preprocessor),
        ("lr", LinearRegression())
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    mae  = float(mean_absolute_error(y_test, y_pred))
    r2   = float(r2_score(y_test, y_pred))

    return rmse, mae, r2

def run_scenario(df, scenario_name, test_year=2024, seed=42):
    feats = SCENARIOS[scenario_name]["features"]
    cats  = SCENARIOS[scenario_name]["cat_cols"]

    X = df[feats].copy()
    y = np.log(df["valor_oferta"])

    is_test = (df["ano"] == test_year)
    X_train, X_test = X.loc[~is_test].copy(), X.loc[is_test].copy()
    y_train, y_test = y.loc[~is_test].copy(), y.loc[is_test].copy()

    # garantir categóricas como string
    for c in cats:
        X_train[c] = X_train[c].astype(str)
        X_test[c]  = X_test[c].astype(str)

    cat_idx = [X_train.columns.get_loc(c) for c in cats]

    model = CatBoostRegressor(
        loss_function="RMSE",
        iterations=10,
        learning_rate=0.05,
        depth=8,
        random_seed=seed,
        verbose=False
    )

    model.fit(
        X_train, y_train,
        cat_features=cat_idx,
        eval_set=(X_test, y_test),
        use_best_model=True
    )

    y_pred = model.predict(X_test)
    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    mae  = float(mean_absolute_error(y_test, y_pred))
    r2   = float(r2_score(y_test, y_pred))

    return rmse, mae, r2

linear_feats = BASE_CAT + BASE_NUM
linear_cats  = BASE_CAT

rmse, mae, r2 = run_linear_baseline(df, linear_feats, linear_cats)

baseline_row = {
    "model": "LinearRegression", 
    "scenario": "linear_baseline",
    "rmse_log": rmse,
    "mae_log": mae,
    "r2": r2
}

rows = []
for name in SCENARIOS.keys():
    rmse, mae, r2 = run_scenario(df, name)
    rows.append({"model": modelselected, "scenario": name, "rmse_log": rmse, "mae_log": mae, "r2": r2})

results = pd.DataFrame(rows).sort_values("r2", ascending=False)

results = pd.concat(
    [pd.DataFrame([baseline_row]), results],
    ignore_index=True
)

results.sort_values("r2", ascending=False)


,model,scenario,rmse_log,mae_log,r2
1,CatBoostRegressor,predictive_bairro,0.410581,0.276058,0.516975
2,CatBoostRegressor,predictive_city,0.413198,0.277583,0.510798
3,CatBoostRegressor,predictive_uf,0.415450,0.279301,0.505450
4,CatBoostRegressor,structural_bairro,0.509801,0.350848,0.255314
5,CatBoostRegressor,structural_city,0.511048,0.352955,0.251665
0,LinearRegression,linear_baseline,0.521777,0.376087,0.219914
6,CatBoostRegressor,structural_uf,0.526383,0.364910,0.206083
